This is a script that performs all the pre-processing necessary for the all-sky source search and source class search of neutrino flux from IceCube's muon track data. The majority of the pre-processing is simply moving fixed width text files to numpy pickle files for quicker reading. The more time consuming pre-processing performed is the background PDF calculation and the integration of IceCube's detector effective area over energy and a neutrino flux. 

First, I load the IceCube track data into a pickle file. 

In [1]:
import glob
import A01_open_and_convert_icecube_data
    
raw_icecube_file_names = glob.glob("./data/3year-data-release/IC*-events.txt")
output_file_name = "processed_data/output_icecube_data.npz"
A01_open_and_convert_icecube_data.main(raw_icecube_file_names, output_file_name)

Loading filename: ./data/3year-data-release/IC86-2012-events.txt
Loading filename: ./data/3year-data-release/IC79-2010-events.txt
Loading filename: ./data/3year-data-release/IC86-2011-events.txt


Next, I calculate the background PDF back scrambling the IceCube track data in RA in $\pm3^\circ$ bands in declination. 

In [2]:
import A02_analyze_background

icecube_file_name = output_file_name
output_file_name = "processed_data/output_icecube_background_count.npz"
sweep_dec, B_i = A02_analyze_background.main(icecube_file_name, output_file_name)

/Users/danielsmith/Documents/icecube/2022_04_04_icecube_source_analysis/3yr_icecube_source_search/A02_analyze_background.py:58: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  sweep_counts_norm, err = scipy.integrate.quad(f_integrand,


The next block simply loads the 4LAC catalog FITS file and saves it to a numpy pickle file. 

In [3]:
import A04p1_open_and_convert_4LAC_catalog

fits_file_name = "./data/table_4LAC.fits"
output_file_name = "./processed_data/4LAC_catelogy.npz"
A04p1_open_and_convert_4LAC_catalog.open_and_convert_catalog(fits_file_name, output_file_name)

The next block is by far the most time consuming (~1 min), and loads IceCube's detector effective volume, averages for detector type (number of strings used for that given year of deployment), and integrates over the following:

$$
A'_{eff}(\delta) = \int_{E_{min}}^{E_{max}} E'^{-\alpha} A_{eff}(E', \delta) dE'
$$

Ultimately, this is will be used to calculate the number of neutrino from a source, $N_\nu$, from a source class using the integral,

$$ 
N_\nu = \int dt \int d\Omega \int_0^\infty dE' A_{eff}(E', \delta) \phi_\nu(E_\nu, \Omega, t).
$$

More description of the process is available both in the paper and in the `README.md` of the GitHub repository.

Two different spectral indices are computed, $\alpha=2.0$ and $2.5$.

In [4]:
import A04p2_open_and_convert_icecube_Aeff

input_file_names = glob.glob("./data/3year-data-release/*Aeff.txt")

for alpha in [2.0, 2.5]:
    output_file_name = "./processed_data/output_icecube_AffIntegrated_%s.npz" % alpha
    dec_steps, y_integrate_steps = A04p2_open_and_convert_icecube_Aeff.load_Aeff(input_file_names, output_file_name, alpha)

/Users/danielsmith/Documents/icecube/2022_04_04_icecube_source_analysis/3yr_icecube_source_search/A04p2_open_and_convert_icecube_Aeff.py:77: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  integrated_Aeff, int_Aeff_error = scipy.integrate.quad(f_integrand,
/Users/danielsmith/Documents/icecube/2022_04_04_icecube_source_analysis/3yr_icecube_source_search/A04p2_open_and_convert_icecube_Aeff.py:77: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  integrated_Aeff, int_Aeff_error = scipy.integrate.quad(f_integrand,
/Users/danielsmith/Documents/icecube/2022_04_04_icecube_source_analysis/3yr_icecube_source_search/A04p2_open_and_convert_icecube_Aeff.py:77: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integrated_Aeff, int_Aeff